In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/logistics-data.csv")
df.columns = df.columns.str.strip()

def to_number(x):
    if pd.isna(x):
        return np.nan
    x = str(x).replace("\u202f", "").replace(" ", "")
    x = x.replace(",", ".")
    try:
        return float(x)
    except:
        return np.nan

for col in ["PLF", "CFX", "TOTAL"]:
    df[col] = df[col].apply(to_number)

df = df.sort_values(["INDICATEUR", "SOUS-INDICATEUR", "ANNEE"])

def interpolate_group(g):
    indicateur, sous_indicateur = g.name

    # --- Proportion PLF dans (PLF+CFX) quand on l'a (pour répartir TOTAL si besoin)
    ratio = np.nan
    mask_known = g["PLF"].notna() & g["CFX"].notna() & ((g["PLF"] + g["CFX"]) > 0)
    if mask_known.any():
        ratio = (g.loc[mask_known, "PLF"] / (g.loc[mask_known, "PLF"] + g.loc[mask_known, "CFX"])).median()
    if np.isnan(ratio):
        ratio = 0.5  # fallback

    g = g.set_index("ANNEE")
    years = range(g.index.min(), g.index.max() + 1)
    g = g.reindex(years)

    g["INDICATEUR"] = indicateur
    g["SOUS-INDICATEUR"] = sous_indicateur

    # 1) Si TOTAL existe et CFX manquant -> CFX = TOTAL - PLF
    m = g["TOTAL"].notna() & g["PLF"].notna() & g["CFX"].isna()
    g.loc[m, "CFX"] = g.loc[m, "TOTAL"] - g.loc[m, "PLF"]

    # 2) Si TOTAL existe et PLF manquant -> PLF = TOTAL - CFX
    m = g["TOTAL"].notna() & g["CFX"].notna() & g["PLF"].isna()
    g.loc[m, "PLF"] = g.loc[m, "TOTAL"] - g.loc[m, "CFX"]

    # 3) Si TOTAL existe et PLF+CFX manquent -> répartir via ratio
    m = g["TOTAL"].notna() & g["PLF"].isna() & g["CFX"].isna()
    g.loc[m, "PLF"] = g.loc[m, "TOTAL"] * ratio
    g.loc[m, "CFX"] = g.loc[m, "TOTAL"] * (1 - ratio)

    # 4) Interpolation linéaire
    g["PLF"] = g["PLF"].interpolate(method="linear", limit_direction="both")
    g["CFX"] = g["CFX"].interpolate(method="linear", limit_direction="both")

    # 5) Compléter TOTAL si manquant
    mt = g["TOTAL"].isna()
    g.loc[mt, "TOTAL"] = g.loc[mt, "PLF"] + g.loc[mt, "CFX"]

    return g.reset_index().rename(columns={"index": "ANNEE"})

df_interpolated = (
    df.groupby(["INDICATEUR", "SOUS-INDICATEUR"], group_keys=False)
      .apply(interpolate_group)
)

print("NaN restants :\n", df_interpolated[["PLF","CFX","TOTAL"]].isna().sum())
df_interpolated.to_csv("data/logistics-data-interpolated.csv", index=False)
print("Fichier sauvegardé ✅")

NaN restants :
 PLF      0
CFX      0
TOTAL    0
dtype: int64
Fichier sauvegardé ✅
